<a href="https://colab.research.google.com/github/kylama/musicrecognition/blob/main/MaSridharMusicCategorizationKNNMultiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

# Import the k-Nearest Neighbors model from the sklearn library
from sklearn.neighbors import KNeighborsClassifier

# Import the train/test data split function from the sklearn library
from sklearn.model_selection import train_test_split

# Importing libraries to save csv file
import os
from google.colab import drive
from numpy import savetxt
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [162]:
df = pd.read_csv('songs_normalize.csv')

xDf = df[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
yDf = df[['genre', 'song', 'artist']]
actualGenre = df['genre']

display(xDf)
display(yDf)
display(actualGenre)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.751,0.834,1,-5.444,0.0437,0.3000,0.000018,0.3550,0.894,95.053
1,0.434,0.897,0,-4.918,0.0488,0.0103,0.000000,0.6120,0.684,148.726
2,0.529,0.496,7,-9.007,0.0290,0.1730,0.000000,0.2510,0.278,136.859
3,0.551,0.913,0,-4.063,0.0466,0.0263,0.000013,0.3470,0.544,119.992
4,0.614,0.928,8,-4.806,0.0516,0.0408,0.001040,0.0845,0.879,172.656
...,...,...,...,...,...,...,...,...,...,...
1995,0.842,0.734,1,-5.065,0.0588,0.0427,0.000000,0.1060,0.952,137.958
1996,0.552,0.702,9,-5.707,0.1570,0.1170,0.000021,0.1050,0.564,169.994
1997,0.847,0.678,9,-8.635,0.1090,0.0669,0.000000,0.2740,0.811,97.984
1998,0.741,0.520,8,-7.513,0.0656,0.4500,0.000002,0.2220,0.347,102.998


,genre,song,artist
0,pop,Oops!...I Did It Again,Britney Spears
1,"rock, pop",All The Small Things,blink-182
2,"pop, country",Breathe,Faith Hill
3,"rock, metal",It's My Life,Bon Jovi
4,pop,Bye Bye Bye,*NSYNC
...,...,...,...
1995,pop,Sucker,Jonas Brothers
1996,pop,Cruel Summer,Taylor Swift
1997,"hip hop, country",The Git Up,Blanco Brown
1998,pop,Dancing With A Stranger (with Normani),Sam Smith


,genre
0,pop
1,"rock, pop"
2,"pop, country"
3,"rock, metal"
4,pop
...,...
1995,pop
1996,pop
1997,"hip hop, country"
1998,pop


In [163]:
y_relationship = np.reshape(yDf['genre'], (len(yDf), ))
for i, value in enumerate(yDf['genre']):
  x = value.split(", ")
  value = x[0]
  y_relationship[i] = value
print(y_relationship[0:15])

['pop' 'rock' 'pop' 'rock' 'pop' 'hip hop' 'hip hop' 'pop' 'pop'
 'Dance/Electronic' 'pop' 'pop' 'pop' 'rock' 'pop']


In [164]:
actual_relationship = np.reshape(actualGenre, (len(actualGenre), ))
for i, value in enumerate(actualGenre):
  x = value.split(", ")
  actual_relationship[i] = x
print(actual_relationship[0:15])

[list(['pop']) list(['rock', 'pop']) list(['pop', 'country'])
 list(['rock', 'metal']) list(['pop']) list(['hip hop', 'pop', 'R&B'])
 list(['hip hop']) list(['pop', 'rock']) list(['pop', 'R&B'])
 list(['Dance/Electronic']) list(['pop']) list(['pop']) list(['pop'])
 list(['rock', 'pop']) list(['pop', 'Dance/Electronic'])]


In [165]:
# Function for transforming genre data (y) into numerical data

for i, element in enumerate(y_relationship):
  if element == 'pop':
    yDf['genre'][i] = 1
  elif element == 'hip hop':
    yDf['genre'][i] = 2
  elif element == 'rock':
    yDf['genre'][i] = 3
  elif element == 'country':
    yDf['genre'][i] = 4
  elif element == 'metal':
    yDf['genre'][i] = 5
  elif element == 'R&B':
    yDf['genre'][i] = 6
  elif element == 'Dance/Electronic':
    yDf['genre'][i] = 7
  elif element == 'Folk/Acoustic':
    yDf['genre'][i] = 8
  elif element == 'latin':
    yDf['genre'][i] = 9
  elif element == 'blues':
    yDf['genre'][i] = 10
  elif element == 'easy listening':
      yDf['genre'][i] = 11
  elif element == 'jazz':
      yDf['genre'][i] = 12
  elif element == 'World/Traditional':
      yDf['genre'][i] = 13
  elif element == 'classical':
      yDf['genre'][i] = 14
  else:
    yDf['genre'][i] = 0

print(yDf)
print(yDf.shape)

<ipython-input-165-36325e29a06e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 1
<ipython-input-165-36325e29a06e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 3
<ipython-input-165-36325e29a06e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 2
<ipython-input-165-36325e29a06e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

     genre                                    song          artist
0        1                  Oops!...I Did It Again  Britney Spears
1        3                    All The Small Things       blink-182
2        1                                 Breathe      Faith Hill
3        3                            It's My Life        Bon Jovi
4        1                             Bye Bye Bye          *NSYNC
...    ...                                     ...             ...
1995     1                                  Sucker  Jonas Brothers
1996     1                            Cruel Summer    Taylor Swift
1997     2                              The Git Up    Blanco Brown
1998     1  Dancing With A Stranger (with Normani)       Sam Smith
1999     2                                 Circles     Post Malone

[2000 rows x 3 columns]
(2000, 3)


<ipython-input-165-36325e29a06e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 11


In [166]:
copyActualGenre = [None]*2000
display(actual_relationship[0:15])
for lst in range(len(actual_relationship)):
  currSample = [None]*len(actual_relationship[lst])
  for val in range(len(actual_relationship[lst])):
    if actual_relationship[lst][val] == 'pop':
      currSample[val] = 1
    elif actual_relationship[lst][val] == 'hip hop':
      currSample[val] = 2
    elif actual_relationship[lst][val] == 'rock':
      currSample[val] = 3
    elif actual_relationship[lst][val] == 'country':
      currSample[val] = 4
    elif actual_relationship[lst][val] == 'metal':
      currSample[val] = 5
    elif actual_relationship[lst][val] == 'R&B':
      currSample[val] = 6
    elif actual_relationship[lst][val] == 'Dance/Electronic':
      currSample[val] = 7
    elif actual_relationship[lst][val] == 'Folk/Acoustic':
      currSample[val] = 8
    elif actual_relationship[lst][val] == 'latin':
      currSample[val] = 9
    elif actual_relationship[lst][val] == 'blues':
      currSample[val] = 10
    elif actual_relationship[lst][val] == 'easy listening':
      currSample[val] = 11
    elif actual_relationship[lst][val] == 'jazz':
      currSample[val] = 12
    elif actual_relationship[lst][val] == 'World/Traditional':
      currSample[val] = 13
    elif actual_relationship[lst][val] == 'classical':
      currSample[val] = 14
    else:
      currSample[val] = 0
  copyActualGenre[lst] = currSample

print(copyActualGenre)


array([list(['pop']), list(['rock', 'pop']), list(['pop', 'country']),
       list(['rock', 'metal']), list(['pop']),
       list(['hip hop', 'pop', 'R&B']), list(['hip hop']),
       list(['pop', 'rock']), list(['pop', 'R&B']),
       list(['Dance/Electronic']), list(['pop']), list(['pop']),
       list(['pop']), list(['rock', 'pop']),
       list(['pop', 'Dance/Electronic'])], dtype=object)

[[1], [3, 1], [1, 4], [3, 5], [1], [2, 1, 6], [2], [1, 3], [1, 6], [7], [1], [1], [1], [3, 1], [1, 7], [2, 1, 6], [1], [1], [1], [2], [3, 5], [3, 8, 11], [1], [2], [1, 7], [5], [1, 7], [2, 1, 6], [1, 7], [7], [2, 1], [1, 6], [1], [6], [1], [2, 1, 6], [1, 7], [2], [1, 4], [1, 6], [1, 9], [1, 6], [2, 1, 6], [2], [2, 1, 6], [1], [1], [1, 6], [6], [2, 1, 6], [1, 6], [1, 9], [2, 1, 6], [2, 1, 6], [2], [1, 6], [6], [1, 6], [1, 4], [8, 3], [1, 11, 7], [1], [1, 6], [2, 1, 6], [1], [1], [3], [2, 1, 6], [2, 1, 6], [3, 10, 9], [1, 6], [1], [1], [1, 3], [7], [1, 6], [1], [1, 3, 5], [1, 6], [2, 1], [3, 1, 5], [1, 6], [1], [1], [1], [1, 6], [1], [2, 6], [1, 9], [1, 3], [1, 6], [1], [1, 3, 5], [1, 8], [1, 6], [0], [1], [1, 3, 5], [1], [2, 1, 6], [2, 1], [1, 6], [2], [1, 7], [1], [2, 1, 9], [1, 9], [3, 1], [1], [1], [1, 6], [1], [1, 7], [2, 1, 6], [1, 6], [1, 7], [5], [1], [7], [1], [2, 1, 6], [1], [1, 6], [3, 5], [1, 6], [2, 1, 6], [2, 1, 6], [2, 1, 6], [3, 5], [2, 7], [2, 1], [1, 7], [1, 6], [1], [1

In [167]:
copy = pd.DataFrame(yDf)
copy['genre'].unique()

array([1, 3, 2, 7, 5, 6, 8, 0, 13, 9, 4, 11], dtype=object)

In [168]:
# Creating array to save to csv file to display on web app
csv_array = []

In [169]:
def num_to_genre(holder):
  if holder == 1:
    holder = 'pop'
  elif holder == 2:
    holder = 'hip hop'
  elif holder == 3:
    holder = 'rock'
  elif holder == 4:
    holder = 'country'
  elif holder == 5:
    holder = 'metal'
  elif holder == 6:
    holder = 'R&B'
  elif holder == 7:
    holder = 'Dance/Electronics'
  elif holder == 8:
    holder = 'Folk/Acoustic'
  elif holder == 9:
    holder = 'latin'
  elif holder == 10:
    holder = 'blues'
  elif holder == 11:
    holder = 'easy listening'
  elif holder == 12:
    holder = 'jazz'
  elif holder == 13:
    holder = 'World/Traditional'
  elif holder == 14:
    holder = 'classical'
  else:
    holder = 'none'
  return holder

In [170]:
def KNN(k):
  accuracy = [] # declare an empty list to hold the accuracy
  for i in range(1):
    X_train, X_test, y_train, y_test, song_train, song_test, artist_train, artist_test, genre_train, genre_test = train_test_split(xDf, yDf['genre'], yDf['song'], yDf['artist'],copyActualGenre, test_size = 0.2, random_state = i) # split the data into train/test split
    y_train = np.array(y_train.astype(int))
    y_test = np.array(y_test.astype(int))
    song_train = np.array(song_train.astype(str))
    song_test = np.array(song_test.astype(str))
    artist_train = np.array(artist_train.astype(str))
    artist_test = np.array(artist_test.astype(str))

    scaler = StandardScaler() # standardize the data
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = KNeighborsClassifier(n_neighbors = k, weights = "distance") # train a KNN model using the training data and labels
    model.fit(X_train_scaled, y_train)
    y_hat = model.predict(X_test_scaled)

  y_pred = model.predict(X_test)
  print(X_test)

  for i in range(len(y_test)):
    song_array = []
    true = y_test[i]
    pred = y_pred[i]
    song = song_test[i]
    artist = artist_test[i]
    print("Song Title: ", song)
    song_array.append(song)
    print("Artist: ", artist)
    song_array.append(artist)
    print("True: ", num_to_genre(true))
    song_array.append(num_to_genre(true))
    print("Pred: ", num_to_genre(pred))
    song_array.append(num_to_genre(pred))
    print("")
    if y_pred[i] in genre_test[i]:
      score = 1 # calculate the accuracy of the model using the testing data
      accuracy.append(score) # append the accuracy to the list containing all accuracies;
    else:
      score = 0 # calculate the accuracy of the model using the testing data
      accuracy.append(score) # append the accuracy to the list containing all accuracies;
    csv_array.append(song_array)

  print(csv_array)

  avg_accuracy = sum(accuracy) / len(accuracy) # calculate the average accuracy by averaging all elements in the list holding all accuracies
  print("average accurcy of k =", k, ":", avg_accuracy)
  return model

In [171]:
user_model = KNN(40)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


      danceability  energy  key  loudness  speechiness  acousticness  \
405          0.663   0.746    2    -3.567       0.0321       0.06970   
1190         0.560   0.936    3    -5.835       0.0439       0.00847   
1132         0.310   0.926    6    -1.131       0.0464       0.01840   
731          0.918   0.857    3    -5.032       0.0623       0.16600   
1754         0.910   0.444    1    -8.126       0.3440       0.02200   
...            ...     ...  ...       ...          ...           ...   
638          0.891   0.439    7    -7.994       0.4280       0.03510   
360          0.893   0.740   11    -4.936       0.1320       0.29900   
1810         0.699   0.713    9    -5.507       0.0594       0.04000   
1743         0.698   0.882    4    -3.078       0.0863       0.12700   
563          0.636   0.558    7    -7.076       0.0245       0.46200   

      instrumentalness  liveness  valence    tempo  
405           0.000000    0.0929    0.325  102.847  
1190          0.000000    0.1

In [172]:
with open('songs.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerows(csv_array)

In [174]:
input_song = np.array([[0.585, 0.842, 9, -5.883, 0.0556, 0.00242, 0.00686, 0.0866, 0.428, 118.211]])
num_to_genre(user_model.predict(input_song))

'pop'